In [ ]:
#pubmed api
search = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&'
'''
# txt 파일을 통한 쿼리 입력
with open("term.txt", "r") as term_file:
    query = term_file.read().strip()
'''

# case1
#query = '(Monkeypox virus[Title/Abstract] OR monkeypox virus[Title/Abstract] OR monkeypox*[Title/Abstract] OR MPXV[Title/Abstract] OR Orthopoxvirus[Title/Abstract] OR Orthopox*[Title/Abstract] OR OPXV[Title/Abstract]) AND (PCR[Title/Abstract] OR polymerase chain reaction*[Title/Abstract] OR real time*[Title/Abstract] OR real-time PCR[Title/Abstract] OR real time PCR[Title/Abstract]) AND (primer[All Fields] OR probe[All Fields] OR molecule[All Fields] OR diagnostic[All Fields])'
# case2
query = '(Shigella[All Fields] OR Shigella spp.[All Fields] OR Shigella sp.[All Fields]) AND (PCR[TIAB] OR polymerase chain reaction*[TIAB] OR real time*[TIAB] OR real-time PCR[TIAB]) AND (primer[All Fields] OR probe[All Fields]) AND (molecular[All Fields] OR diagnostic[All Fields])'

retmax = "100"
url=search+"term="+query+"&retmode=json&retmax="+retmax


In [ ]:
print(url)

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&term=(Shigella[All Fields] OR Shigella spp.[All Fields] OR Shigella sp.[All Fields]) AND (PCR[TIAB] OR polymerase chain reaction*[TIAB] OR real time*[TIAB] OR real-time PCR[TIAB]) AND (primer[All Fields] OR probe[All Fields]) AND (molecular[All Fields] OR diagnostic[All Fields])&retmode=json&retmax=100


In [ ]:
import pprint
import requests
data = requests.get(url).json()
pprint.pprint(data)

{'esearchresult': {'count': '19250',
                   'errorlist': {'fieldsnotfound': [], 'phrasesnotfound': []},
                   'idlist': ['11342756',
                              '11342206',
                              '11341105',
                              '11340603',
                              '11340327',
                              '11336092',
                              '11326835',
                              '11326090',
                              '11324020',
                              '11323537',
                              '11323402',
                              '11322368',
                              '11321392',
                              '11319344',
                              '11319304',
                              '11318556',
                              '11316331',
                              '11315647',
                              '11315073',
                              '11313837',
                              '11313783',
  

In [ ]:
import pandas as pd
df = pd.DataFrame(data["esearchresult"]["idlist"])
df.columns = ['id']
print(df)

          id
0   11342756
1   11342206
2   11341105
3   11340603
4   11340327
..       ...
95  11342760
96  11341846
97  11340806
98  11340127
99  11339456

[100 rows x 1 columns]


# xml download

In [ ]:
import requests
import pandas as pd
import time
import os
from bs4 import BeautifulSoup

In [ ]:
records = []
# efetch api
import xml.etree.ElementTree as ET
for pmcid in df['id']:
    efetch_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id={pmcid}&rettype=xml'
    print(efetch_url)
    fetch_response = requests.get(efetch_url, params={'db': 'pmc', 'id': pmcid, 'rettype': 'xml'})
    if fetch_response.status_code == 200:
        try:
            # parse xml to extract pmcid, doi, introduction
            root = ET.fromstring(fetch_response.content)
            # extract doi
            doi = root.findtext('.//article-id[@pub-id-type="doi"]')
            if doi is None:
                doi = 'N/A'
            # extract introduction
            introduction = root.find(".//sec[@sec-type='intro']") or root.find(".//sec[title='introduction']")
            if introduction is not None:
                intro_text = ''.join(introduction.itertext()).strip().replace('\n', ' ')
            else:
                intro_text = 'N/A'

            response = {
                'pmcid': pmcid,
                'doi': doi,
                'introduction': intro_text
            }
            print(response)
            records.append(response)

        except ET.ParseError:
             print(f'Failed to parse XML for PMCID {pmcid}')
    else:
        print(f'Failed to fetch data for PMCID {pmcid}. Status code: {fetch_response.status_code}')
        continue

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=11342756&rettype=xml
{'pmcid': '11342756', 'doi': '10.1016/j.psj.2024.104099', 'introduction': 'N/A'}
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=11342206&rettype=xml
{'pmcid': '11342206', 'doi': '10.1016/j.bioactmat.2024.07.032', 'introduction': 'N/A'}
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=11341105&rettype=xml
{'pmcid': '11341105', 'doi': '10.7759/cureus.65185', 'introduction': 'Introduction       Pseudomonas aeruginosa is an opportunistic pathogen responsible for various healthcare-related infections [1]. These infections are challenging to treat due to the bacteria\'s intrinsic and acquired resistance mechanisms [2]. The improper and unnecessary use of β-lactam antibiotics exacerbates bacterial resistance, leading to the selection of extended-spectrum β-lactamases (ESBL), AmpC β-lactamases, and metallo-β-lactamases (MBL), which pose significant challenges in anti

In [ ]:
print(records)
print(len(records))

[{'pmcid': '11342756', 'doi': '10.1016/j.psj.2024.104099', 'introduction': 'N/A'}, {'pmcid': '11342206', 'doi': '10.1016/j.bioactmat.2024.07.032', 'introduction': 'N/A'}, {'pmcid': '11341105', 'doi': '10.7759/cureus.65185', 'introduction': 'Introduction       Pseudomonas aeruginosa is an opportunistic pathogen responsible for various healthcare-related infections [1]. These infections are challenging to treat due to the bacteria\'s intrinsic and acquired resistance mechanisms [2]. The improper and unnecessary use of β-lactam antibiotics exacerbates bacterial resistance, leading to the selection of extended-spectrum β-lactamases (ESBL), AmpC β-lactamases, and metallo-β-lactamases (MBL), which pose significant challenges in antimicrobial therapy [3]. Pseudomonas aeruginosa, typically susceptible to ceftazidime, aztreonam, and carboxypenicillins, can develop resistance to third-generation cephalosporins through the hyperproduction of AmpC beta-lactamase [2]. AmpC β-lactamases hydrolyze ce

In [ ]:
for item in records:
    pmcid = item.get('pmcid')
    if pmcid:
        fetch_response = requests.get(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id={pmcid}&retmode=xml')
        if fetch_response.status_code==200:
            file_name = f'{pmcid}.xml'
            with open(file_name, "w") as file:
                file.write(fetch_response.text)
            print(f"saved {pmcid} to {file_name}")
        else:
            print(f"Failed to fetch data for {pmcid}. Status code: {fetch_response.status_code}")

saved 11342756 to 11342756.xml
saved 11342206 to 11342206.xml
saved 11341105 to 11341105.xml
Failed to fetch data for 11340603. Status code: 429
Failed to fetch data for 11340327. Status code: 429
Failed to fetch data for 11326835. Status code: 429
Failed to fetch data for 11324020. Status code: 429
Failed to fetch data for 11323402. Status code: 429
Failed to fetch data for 11322368. Status code: 429
saved 11319344 to 11319344.xml
saved 11318556 to 11318556.xml
saved 11316331 to 11316331.xml
Failed to fetch data for 11315073. Status code: 429
Failed to fetch data for 11313837. Status code: 429
Failed to fetch data for 11313783. Status code: 429
Failed to fetch data for 11312530. Status code: 429
Failed to fetch data for 11312258. Status code: 429
Failed to fetch data for 11312129. Status code: 429
saved 11311926 to 11311926.xml
Failed to fetch data for 11311727. Status code: 429
saved 11304630 to 11304630.xml
Failed to fetch data for 11304591. Status code: 400
saved 11303641 to 113036

In [ ]:
pmcid_list = []
for item in records:
    pmcid = item.get('pmcid')
    if pmcid:
        pmcid_list.append(pmcid)

xml_count = len([f for f in os.listdir('/content/') if f.endswith('.xml')])

print(f'numbe of pmcid : {len(pmcid_list)}')
print(f'number of xml : {xml_count}')

numbe of pmcid : 59
number of xml : 25


In [ ]:
# extracted data to tsv file
df = pd.DataFrame(records)
df.head()

,pmcid,doi,introduction
0,11342756,10.1016/j.psj.2024.104099,N/A
1,11342206,10.1016/j.bioactmat.2024.07.032,N/A
2,11341105,10.7759/cureus.65185,Introduction Pseudomonas aeruginosa is a...
3,11340603,10.1016/j.csbj.2024.07.017,N/A
4,11340327,10.1128/jb.00049-24,INTRODUCTION The gram-negative bacterial...


In [ ]:
# PMCID, DOI, Introduction 포함
df.to_csv('articles.tsv', sep='\t', index=False)

In [ ]:
!zip articles_xml.zip *.xml

  adding: 11240451.xml (deflated 79%)
  adding: 11244529.xml (deflated 82%)
  adding: 11246856.xml (deflated 81%)
  adding: 11249912.xml (deflated 80%)
  adding: 11262882.xml (deflated 78%)
  adding: 11263920.xml (deflated 82%)
  adding: 11265448.xml (deflated 80%)
  adding: 11273848.xml (deflated 78%)
  adding: 11279407.xml (deflated 82%)
  adding: 11285316.xml (deflated 76%)
  adding: 11289428.xml (deflated 82%)
  adding: 11300865.xml (deflated 77%)
  adding: 11302304.xml (deflated 78%)
  adding: 11302663.xml (deflated 80%)
  adding: 11303641.xml (deflated 83%)
  adding: 11304630.xml (deflated 81%)
  adding: 11311926.xml (deflated 78%)
  adding: 11316331.xml (deflated 76%)
  adding: 11318556.xml (deflated 81%)
  adding: 11319344.xml (deflated 78%)
  adding: 11340806.xml (deflated 78%)
  adding: 11341105.xml (deflated 76%)
  adding: 11341846.xml (deflated 80%)
  adding: 11342206.xml (deflated 81%)
  adding: 11342756.xml (deflated 80%)


# pdf download

In [ ]:
pdf_dir = '/content/PMC_pdfs'
if not os.path.exists(pdf_dir):
    os.makedirs(pdf_dir)

In [ ]:
pmcid_list = []
for item in records:
    pmcid = item.get('pmcid')
    if pmcid:
        pmcid_list.append(pmcid)
print(pmcid_list)

['11342756', '11342206', '11341105', '11340603', '11340327', '11326835', '11324020', '11323402', '11322368', '11319344', '11318556', '11316331', '11315073', '11313837', '11313783', '11312530', '11312258', '11312129', '11311926', '11311727', '11304630', '11304591', '11303641', '11302663', '11302304', '11300865', '11237486', '11293832', '11293673', '11291472', '11289428', '11288818', '11285316', '11281493', '11279407', '11278787', '11273848', '11272550', '11269248', '11265448', '11263920', '11262882', '11261454', '11260827', '11257890', '11249912', '11246856', '11244529', '11243892', '11243103', '11242183', '11240787', '11240451', '11238125', '11343591', '11341846', '11340806', '11340127', '11339456']


In [ ]:
print(len(pmcid_list))

59


In [ ]:
records = []
# Europe PMC API URL
for PMCID in pmcid_list:
    europe_pmc_url = f'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PMCID:PMC{PMCID}&resultType=core&format=json'

    try:
        response = requests.get(europe_pmc_url, timeout=10)

        if response.status_code == 200:
            print(europe_pmc_url)
            result_json = response.json()

            for article in result_json.get("resultList", {}).get("result", []):
                record = {
                    'pmcid': article.get('pmcid', None),
                    'pmid': article.get('pmid', None),
                    'doi': article.get('doi', None),
                    'title': article.get('title', None),
                    'abstractText': article.get('abstractText', None),
                    'pdf_urls': None,
                    'printPublicationDate': article.get('journalInfo', {}).get('printPublicationDate', None),
                    'dateOfCreation': article.get('dateOfCreation', None),
                    'firstIndexDate': article.get('firstIndexDate', None),
                    'fullTextReceivedDate': article.get('fullTextReceivedDate', None),
                    'dateOfRevision': article.get('dateOfRevision', None),
                    'electronicPublicationDate': article.get('electronicPublicationDate', None),
                    'firstPublicationDate': article.get('firstPublicationDate', None),
                    'citedByCount': article.get('citedByCount', None)
                }

                # Open Access PDF URL 추출
                if article.get("isOpenAccess", False):
                    pdf_link = article.get("fullTextUrlList", {}).get("fullTextUrl", [])
                    pdf_list = [link.get("url") for link in pdf_link if link.get("documentStyle", "") == "pdf"
                                and (link.get("availability", "") == "Open access"
                                     or link.get("availability", "") == "Free")]
                    if pdf_list:
                        record['pdf_urls'] = pdf_list

                        # timeout / error handling
                        for attempt, url in enumerate(record['pdf_urls']):
                            headers = {
                                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
                            }
                            try:
                                pdf_response = requests.get(url, headers=headers, verify=False, timeout=10)
                                if pdf_response.status_code == 200:
                                    pdf_path = os.path.join(pdf_dir, f"PMCID_{PMCID}.pdf")
                                    with open(pdf_path, 'wb') as f:
                                        f.write(pdf_response.content)
                                        print(f"Downloaded: {pdf_path}")
                                        break
                                else:
                                    print(f"Failed to download {url}, status code: {pdf_response.status_code}")
                                    time.sleep(2)
                            except requests.exceptions.RequestException as e:
                                print(f"Error occurred for {url}: {e}")
                                time.sleep(2)

                records.append(record)

    except requests.exceptions.Timeout as e:
        print(f"Timeout error occurred for PMC{PMCID}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Request error occurred for PMC{PMCID}: {e}")

df = pd.DataFrame(records)


https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PMCID:PMC11342756&resultType=core&format=json


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'europepmc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'europepmc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: /content/PMC_pdfs/PMCID_11342756.pdf
https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PMCID:PMC11342206&resultType=core&format=json


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: /content/PMC_pdfs/PMCID_11342206.pdf
https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PMCID:PMC11341105&resultType=core&format=json


...

Downloaded: /content/PMC_pdfs/PMCID_11340806.pdf
https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PMCID:PMC11340127&resultType=core&format=json


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: /content/PMC_pdfs/PMCID_11340127.pdf
https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=PMCID:PMC11339456&resultType=core&format=json


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: /content/PMC_pdfs/PMCID_11339456.pdf


In [ ]:
df.head()

,pmcid,pmid,doi,title,abstractText,pdf_urls,printPublicationDate,dateOfCreation,firstIndexDate,fullTextReceivedDate,dateOfRevision,electronicPublicationDate,firstPublicationDate,citedByCount
0,PMC9493680,36160211,10.3389/fmicb.2022.996794,"Detection of <i>Escherichia coli</i>, <i>Pseud...",To eliminate the influences of excipients and ...,[https://www.frontiersin.org/articles/10.3389/...,2022-01-01,2022-09-26,2022-09-28,2022-09-24,2022-09-28,2022-09-08,2022-09-08,0
1,PMC8414663,34479650,10.1186/s13104-021-05759-6,RAPD and ERIC-PCR coupled with HRM for species...,<h4>Objective</h4>Species identification of Sh...,[https://bmcresnotes.biomedcentral.com/counter...,2021-09-01,2021-09-04,2021-09-05,2021-09-14,2021-09-10,2021-09-03,2021-09-03,1
2,PMC7587089,32817086,10.1128/jcm.01493-20,Additional Og-Typing PCR Techniques Targeting ...,The O-serogrouping of pathogenic <i>Escherichi...,[https://europepmc.org/articles/PMC7587089?pdf...,2020-10-01,2020-08-21,2020-08-21,2021-04-27,2021-06-23,2020-10-21,2020-10-21,13
3,PMC7492956,32963655,10.1155/2020/9373984,Development and Evaluation of an iiPCR Assay f...,<h4>Background</h4><i>Salmonella</i> and <i>Sh...,[https://downloads.hindawi.com/journals/cjidmm...,2020-01-01,2020-09-23,2020-09-24,2020-09-24,2022-04-17,2020-09-07,2020-09-07,0
4,PMC7461712,32884951,10.30498/ijb.2020.127412.2244,Development and Application of an Immunocaptur...,<h4>Background</h4><i>Shigella</i> is among th...,[https://europepmc.org/articles/PMC7461712?pdf...,2020-01-01,2020-09-05,2020-09-05,2020-09-04,2022-04-16,2020-01-01,2020-01-01,2


In [ ]:
len(df)

59

In [ ]:
df['pdf_urls'].notnull().sum()

59

In [ ]:
df.to_csv('europepmc_pmcid_result.csv', index='False')

In [ ]:
!ls -l /content/PMC_pdfs | wc -l

60


In [ ]:
!zip -r /content/PMC_pdfs.zip /content/PMC_pdfs

  adding: content/PMC_pdfs/ (stored 0%)
  adding: content/PMC_pdfs/PMCID_10965209.pdf (deflated 15%)
  adding: content/PMC_pdfs/PMCID_10991388.pdf (deflated 14%)
  adding: content/PMC_pdfs/PMCID_11052509.pdf (deflated 8%)
  adding: content/PMC_pdfs/PMCID_11221667.pdf (deflated 16%)
  adding: content/PMC_pdfs/PMCID_11281682.pdf (deflated 4%)
  adding: content/PMC_pdfs/PMCID_11097627.pdf (deflated 15%)
  adding: content/PMC_pdfs/PMCID_11013254.pdf (deflated 10%)
  adding: content/PMC_pdfs/PMCID_10974990.pdf (deflated 11%)
  adding: content/PMC_pdfs/PMCID_11275270.pdf (deflated 10%)
  adding: content/PMC_pdfs/PMCID_10974247.pdf (deflated 7%)
  adding: content/PMC_pdfs/PMCID_11279579.pdf (deflated 24%)
  adding: content/PMC_pdfs/PMCID_11270334.pdf (deflated 6%)
  adding: content/PMC_pdfs/PMCID_11288998.pdf (deflated 16%)
  adding: content/PMC_pdfs/PMCID_11142709.pdf (deflated 15%)
  adding: content/PMC_pdfs/PMCID_11253681.pdf (deflated 11%)
  adding: content/PMC_pdfs/PMCID_11254000.pdf (de